In [1]:
import sys
import os
import numpy as np

NENSMEMBERS=20
NPARAMETERS=2
PRIOR_ERR=0.5
DA_DIR = '/home/users/ndouglas/TRUTHS/Data_Assimilation_files'

LAI_PRIOR=3.0
cab_PRIOR=90.0

def test_bnds(xbi, val_bnds):

    for xi in enumerate(xbi):
        if val_bnds[xi[0]][0] <= xi[1] <= val_bnds[xi[0]][1]:
            continue
        else:
            return False
    return True


def generate_param_ens(param_dic, size_ens=10, prior_err=0.25):

    p_keys = list(param_dic.keys())
    xb = []
    val_bnds = []
    for param in p_keys:
        xb.append(param_dic[param][0])
        val_bnds.append(param_dic[param][1])
    
    xb = np.array(xb)
    xb_sd = xb * prior_err
    b_mat = np.eye(len(xb))*((xb_sd)**2)

    ens = []
    i = 0
    while len(ens) < size_ens:
        xbi = np.random.multivariate_normal(xb, b_mat)
        i += 1
        if test_bnds(xbi, val_bnds) is True:
            ens.append(xbi)
        else:
            continue
    return np.array(ens)


if __name__ == '__main__':

    prior_params = {
            'a_LAI': [LAI_PRIOR, (0.0, 6.0)],
            'b_cab': [cab_PRIOR, (60.0, 120.0)]
            }
    
    param_ens = generate_param_ens(prior_params, size_ens=NENSMEMBERS,
				prior_err=PRIOR_ERR)
    ens_mean=np.mean(param_ens,0)

    np.save(DA_DIR+'/Xb_semid_20', np.transpose(param_ens))
    np.save(DA_DIR+'/xb_semid_20', np.transpose(ens_mean))
